In [1]:

import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [2]:

seq_length=2
x_data_dim=4
batch_size=100
min_max_normalization_flag=True

data_dir = '../dataset'
fname = os.path.join(data_dir, 'data-02-stock_daily.csv')
df = pd.read_csv(fname)
dataset=df.copy()
ori_Y=dataset.pop("Close")
ori_X=dataset.copy()


In [3]:

X_train, X_test, Y_train, Y_test = train_test_split(ori_X,ori_Y, test_size=0.2, shuffle=False)
X_train, X_val, Y_train, Y_val= train_test_split(X_train,Y_train, test_size=0.2, shuffle=False)



In [4]:

## 데이터의 min , max, mean, std 값 구하기.
dataset_stats = X_train.describe()
dataset_stats = dataset_stats.transpose()

## data normalization
def min_max_norm(x):
  return (x - dataset_stats['min']) / (dataset_stats['max'] - dataset_stats['min'])

def standard_norm(x):
  return (x - dataset_stats['mean']) / dataset_stats['std']

if min_max_normalization_flag==True:
    min_max_norm_train_data = min_max_norm(X_train)
    min_max_norm_val_data = min_max_norm(X_val)
    min_max_norm_test_data = min_max_norm(X_test)

    data_gen_train=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_train_data.values.tolist(), Y_train.values.tolist(),
                                                                        length=seq_length, sampling_rate=1,
                                                                        batch_size=batch_size)
    data_gen_val=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_val_data.values.tolist(), Y_val.values.tolist(),
                                                                       length=seq_length, sampling_rate=1,
                                                                       batch_size=batch_size)
    data_gen_test=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_test_data.values.tolist(), Y_test.values.tolist(),
                                                                       length=seq_length, sampling_rate=1,
                                                                       batch_size=batch_size)
else:
    data_gen_train = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_train.values.tolist(),Y_train.values.tolist(),
                                                                   length=seq_length, sampling_rate=1,
                                                                   batch_size=batch_size)
    data_gen_val = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_val.values.tolist(),Y_val.values.tolist(),
                                                                   length=seq_length, sampling_rate=1,
                                                                   batch_size=batch_size)
    data_gen_test = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_test.values.tolist(),Y_test.values.tolist(),
                                                                        length=seq_length, sampling_rate=1,
                                                                        batch_size=batch_size)


In [5]:

input_Layer = tf.keras.layers.Input(shape=(seq_length, x_data_dim))
x= tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(50,activation='tanh'))(input_Layer)
x=tf.keras.layers.Dense(100,activation='relu')(x)
x=tf.keras.layers.Dense(50,activation='relu')(x)
Out_Layer=tf.keras.layers.Dense(1,activation=None)(x)
model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()


loss_function=tf.keras.losses.mean_squared_error
optimize=tf.keras.optimizers.Adam(learning_rate=0.001)
metric=tf.keras.metrics.mean_absolute_error
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

history = model.fit(
      data_gen_train,
      validation_data=data_gen_val,
      steps_per_epoch=len(X_train)/batch_size,
      epochs=500,
      validation_freq=1
)
print(model.evaluate(data_gen_test))


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 4)]            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 100)               5500      
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 20,701
Trainable params: 20,701
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
5/4 [================================] - 4s 894ms/step - loss: 500479.2500 - mean_absolute_error: 702.37

Epoch 42/500
5/4 [================================] - 1s 102ms/step - loss: 2201.7837 - mean_absolute_error: 38.1166 - val_loss: 1751.0753 - val_mean_absolute_error: 32.9717
Epoch 43/500
5/4 [================================] - 1s 108ms/step - loss: 2184.7380 - mean_absolute_error: 37.8848 - val_loss: 1765.8538 - val_mean_absolute_error: 33.0288
Epoch 44/500
5/4 [================================] - 1s 102ms/step - loss: 2177.1294 - mean_absolute_error: 37.7536 - val_loss: 1780.1600 - val_mean_absolute_error: 33.0747
Epoch 45/500
5/4 [================================] - 1s 139ms/step - loss: 2168.5237 - mean_absolute_error: 37.6551 - val_loss: 1716.9767 - val_mean_absolute_error: 32.0193
Epoch 46/500
5/4 [================================] - 1s 135ms/step - loss: 2144.5317 - mean_absolute_error: 37.4486 - val_loss: 1740.6583 - val_mean_absolute_error: 32.2145
Epoch 47/500
5/4 [================================] - 0s 88ms/step - loss: 2112.6758 - mean_absolute_error: 37.1912 - val_loss: 17

5/4 [================================] - 1s 174ms/step - loss: 1551.6635 - mean_absolute_error: 32.3405 - val_loss: 1305.0709 - val_mean_absolute_error: 27.5914
Epoch 90/500
5/4 [================================] - 1s 163ms/step - loss: 1545.9139 - mean_absolute_error: 32.3479 - val_loss: 1317.5312 - val_mean_absolute_error: 27.8242
Epoch 91/500
5/4 [================================] - 1s 121ms/step - loss: 1530.9879 - mean_absolute_error: 32.3097 - val_loss: 1328.0739 - val_mean_absolute_error: 27.9948
Epoch 92/500
5/4 [================================] - 1s 122ms/step - loss: 1520.4553 - mean_absolute_error: 32.2385 - val_loss: 1310.3093 - val_mean_absolute_error: 27.8303
Epoch 93/500
5/4 [================================] - 1s 113ms/step - loss: 1543.5249 - mean_absolute_error: 32.3065 - val_loss: 1208.4882 - val_mean_absolute_error: 26.5816
Epoch 94/500
5/4 [================================] - 0s 97ms/step - loss: 1492.5835 - mean_absolute_error: 31.6808 - val_loss: 1173.5095 - val

Epoch 136/500
5/4 [================================] - 0s 60ms/step - loss: 861.5645 - mean_absolute_error: 24.4093 - val_loss: 710.0181 - val_mean_absolute_error: 20.7387
Epoch 137/500
5/4 [================================] - 0s 51ms/step - loss: 860.6328 - mean_absolute_error: 24.3416 - val_loss: 628.7947 - val_mean_absolute_error: 19.3966
Epoch 138/500
5/4 [================================] - 0s 53ms/step - loss: 825.6807 - mean_absolute_error: 23.8047 - val_loss: 639.4031 - val_mean_absolute_error: 19.5581
Epoch 139/500
5/4 [================================] - 0s 48ms/step - loss: 854.5804 - mean_absolute_error: 24.2091 - val_loss: 693.1704 - val_mean_absolute_error: 20.5540
Epoch 140/500
5/4 [================================] - 0s 44ms/step - loss: 815.3569 - mean_absolute_error: 23.6845 - val_loss: 617.8049 - val_mean_absolute_error: 19.2014
Epoch 141/500
5/4 [================================] - 0s 65ms/step - loss: 791.3162 - mean_absolute_error: 23.3254 - val_loss: 614.3832 - v

5/4 [================================] - 0s 52ms/step - loss: 429.5454 - mean_absolute_error: 17.2756 - val_loss: 433.5665 - val_mean_absolute_error: 15.8385
Epoch 184/500
5/4 [================================] - 0s 49ms/step - loss: 430.7398 - mean_absolute_error: 17.2338 - val_loss: 352.2624 - val_mean_absolute_error: 13.9102
Epoch 185/500
5/4 [================================] - 0s 60ms/step - loss: 410.5770 - mean_absolute_error: 16.8541 - val_loss: 360.9502 - val_mean_absolute_error: 14.1070
Epoch 186/500
5/4 [================================] - 0s 62ms/step - loss: 408.1908 - mean_absolute_error: 16.8284 - val_loss: 373.6942 - val_mean_absolute_error: 14.4133
Epoch 187/500
5/4 [================================] - 1s 111ms/step - loss: 414.1378 - mean_absolute_error: 17.0849 - val_loss: 465.2885 - val_mean_absolute_error: 16.6043
Epoch 188/500
5/4 [================================] - 1s 117ms/step - loss: 396.4713 - mean_absolute_error: 16.5218 - val_loss: 409.5006 - val_mean_abso

Epoch 231/500
5/4 [================================] - 0s 56ms/step - loss: 279.2997 - mean_absolute_error: 13.6085 - val_loss: 236.0777 - val_mean_absolute_error: 11.1175
Epoch 232/500
5/4 [================================] - 0s 64ms/step - loss: 304.6712 - mean_absolute_error: 14.3523 - val_loss: 295.4481 - val_mean_absolute_error: 12.8476
Epoch 233/500
5/4 [================================] - 0s 61ms/step - loss: 301.0701 - mean_absolute_error: 14.3946 - val_loss: 210.1560 - val_mean_absolute_error: 10.4981
Epoch 234/500
5/4 [================================] - 0s 67ms/step - loss: 293.0027 - mean_absolute_error: 13.8899 - val_loss: 195.7124 - val_mean_absolute_error: 10.1855
Epoch 235/500
5/4 [================================] - 0s 76ms/step - loss: 272.4731 - mean_absolute_error: 13.3264 - val_loss: 249.5014 - val_mean_absolute_error: 11.5763
Epoch 236/500
5/4 [================================] - 0s 67ms/step - loss: 273.0272 - mean_absolute_error: 13.3630 - val_loss: 234.4892 - v

Epoch 279/500
5/4 [================================] - 0s 65ms/step - loss: 219.3719 - mean_absolute_error: 11.7948 - val_loss: 142.8976 - val_mean_absolute_error: 8.7047
Epoch 280/500
5/4 [================================] - 0s 58ms/step - loss: 220.3820 - mean_absolute_error: 11.8905 - val_loss: 161.7272 - val_mean_absolute_error: 9.3515
Epoch 281/500
5/4 [================================] - 0s 49ms/step - loss: 220.6510 - mean_absolute_error: 12.0686 - val_loss: 140.6026 - val_mean_absolute_error: 8.6298
Epoch 282/500
5/4 [================================] - 0s 61ms/step - loss: 206.8881 - mean_absolute_error: 11.6239 - val_loss: 165.0558 - val_mean_absolute_error: 9.4662
Epoch 283/500
5/4 [================================] - 0s 61ms/step - loss: 203.9116 - mean_absolute_error: 11.4631 - val_loss: 185.1001 - val_mean_absolute_error: 10.2063
Epoch 284/500
5/4 [================================] - 0s 69ms/step - loss: 207.2650 - mean_absolute_error: 11.5876 - val_loss: 149.0039 - val_m

5/4 [================================] - 0s 47ms/step - loss: 119.4040 - mean_absolute_error: 8.4800 - val_loss: 49.9440 - val_mean_absolute_error: 5.4732
Epoch 463/500
5/4 [================================] - 0s 49ms/step - loss: 137.2995 - mean_absolute_error: 9.3598 - val_loss: 86.7744 - val_mean_absolute_error: 7.6741
Epoch 464/500
5/4 [================================] - 0s 58ms/step - loss: 116.1681 - mean_absolute_error: 8.3589 - val_loss: 36.7020 - val_mean_absolute_error: 4.5770
Epoch 465/500
5/4 [================================] - 0s 69ms/step - loss: 120.6337 - mean_absolute_error: 8.4315 - val_loss: 43.1478 - val_mean_absolute_error: 5.0534
Epoch 466/500
5/4 [================================] - 0s 52ms/step - loss: 114.2249 - mean_absolute_error: 8.2100 - val_loss: 42.5660 - val_mean_absolute_error: 5.0123
Epoch 467/500
5/4 [================================] - 0s 48ms/step - loss: 116.5767 - mean_absolute_error: 8.2366 - val_loss: 47.8264 - val_mean_absolute_error: 5.3836


In [ ]:
test_data_X, test_data_Y=data_gen_test[0]
prediction_Y=model.predict(test_data_X).flatten()
Y_test=test_data_Y.flatten()

visual_y=[]
visual_pre_y=[]
for i in range(len(prediction_Y)):
    label = Y_test[i]
    prediction = prediction_Y[i]
    print("실제가격: {:.3f}, 예상가격: {:.3f}".format(label, prediction))
    visual_y.append(label)
    visual_pre_y.append(prediction)

time = range(1, len(visual_y) + 1)
plt.plot(time, visual_y, 'r', label='ture')
plt.plot(time, visual_pre_y, 'b', label='prediction')
plt.title('stock prediction')
plt.xlabel('time')
plt.ylabel('value')
plt.legend()
plt.show()
